In [1]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-28 10:31:06--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.163, 18.239.38.181, 18.239.38.83, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet.1’

yellow_tripdata_202 100%[===================>]  45.46M   114MB/s    in 0.4s    

2024-05-28 10:31:07 (114 MB/s) - ‘yellow_tripdata_2023-01.parquet.1’ saved [47673370/47673370]

--2024-05-28 10:31:07--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.181, 18.239.38.163, 18.239.38.147, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.181|:443... conne

In [4]:
jan_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
print(f"number of columns in january 2023 data: {len(jan_data.columns)}")

number of columns in january 2023 data: 19


In [9]:
jan_data["duration"] = jan_data.tpep_dropoff_datetime - jan_data.tpep_pickup_datetime
jan_data.duration = jan_data.duration.apply(lambda td: td.total_seconds() / 60.0)
print(f"the std of teios duration in january is: {jan_data.duration.std()}")

the std of teios duration in january is: 42.59435124195458


In [13]:
clean_data = jan_data[(jan_data.duration >= 1.0) & (jan_data.duration <=60.0)].copy()
print(f"fraction of original data in cleaned: {(len(clean_data) / len(jan_data))* 100}")

fraction of original data in cleaned: 98.1220282212598


In [16]:
clean_data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [17]:
cat_col = ['PULocationID', 'DOLocationID']
clean_data[cat_col] = clean_data[cat_col].astype(str)
train_dict = clean_data[cat_col].to_dict(orient='records')

In [20]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()

X_train = dv.fit_transform(train_dict)

print(f"the number of columns in training matrix: {X_train.shape[1]}")

the number of columns in training matrix: 515


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y_train = clean_data.duration.values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f"Training RMSE: {mean_squared_error(y_train, y_pred, squared=False)}")

Training RMSE: 7.649261027862345


In [23]:
feb_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

feb_data["duration"] = feb_data.tpep_dropoff_datetime - feb_data.tpep_pickup_datetime
feb_data.duration = feb_data.duration.apply(lambda td: td.total_seconds() / 60.0)
val_data = feb_data[(feb_data.duration >= 1.0) & (feb_data.duration <=60.0)].copy()
val_data[cat_col] = val_data[cat_col].astype(str)

In [24]:
val_dict = val_data[cat_col].to_dict(orient="records")
x_val = dv.transform(val_dict)
y_val = val_data.duration.values

In [25]:
y_pred = lr.predict(x_val)

In [26]:
print(f" Validation RMSE: {mean_squared_error(y_val, y_pred, squared=False)}")

 Validation RMSE: 7.811832772631433
